In [33]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 6: Guard Gallivant ---</h2><p>The Historians use their fancy <a href="4">device</a> again, this time to whisk you all away to the North Pole prototype suit manufacturing lab... in the year <a href="/2018/day/5">1518</a>! It turns out that having direct access to history is very convenient for a group of historians.</p>
<p>You still have to be careful of time paradoxes, and so it will be important to avoid anyone from 1518 while The Historians search for the Chief. Unfortunately, a single <em>guard</em> is patrolling this part of the lab.</p>
<p>Maybe you can work out where the guard will go ahead of time so that The Historians can search safely?</p>
<p>You start by making a map (your puzzle input) of the situation. For example:</p>
<pre><code>....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
</code></pre>
<p>The map shows the current position of the guard with <code>^</code> (to indicate the guard is currently facing <em>up</em> from the perspective of the map). Any <em>obstructions</em> - crates, desks, alchemical reactors, etc. - are shown as <code>#</code>.</p>
<p>Lab guards in 1518 follow a very strict patrol protocol which involves repeatedly following these steps:</p>
<ul>
<li>If there is something directly in front of you, turn right 90 degrees.</li>
<li>Otherwise, take a step forward.</li>
</ul>
<p>Following the above protocol, the guard moves up several times until she reaches an obstacle (in this case, a pile of failed suit prototypes):</p>
<pre><code>....#.....
....^....#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
</code></pre>
<p>Because there is now an obstacle in front of the guard, she turns right before continuing straight in her new facing direction:</p>
<pre><code>....#.....
........&gt;#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
</code></pre>
<p>Reaching another obstacle (a spool of several <em>very</em> long polymers), she turns right again and continues downward:</p>
<pre><code>....#.....
.........#
..........
..#.......
.......#..
..........
.#......v.
........#.
#.........
......#...
</code></pre>
<p>This process continues for a while, but the guard eventually leaves the mapped area (after walking past a tank of universal solvent):</p>
<pre><code>....#.....
.........#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#v..
</code></pre>
<p>By predicting the guard's route, you can determine which specific positions in the lab will be in the patrol path. <em>Including the guard's starting position</em>, the positions visited by the guard before leaving the area are marked with an <code>X</code>:</p>
<pre><code>....#.....
....XXXXX#
....X...X.
..#.X...X.
..XXXXX#X.
..X.X.X.X.
.#XXXXXXX.
.XXXXXXX#.
#XXXXXXX..
......#X..
</code></pre>
<p>In this example, the guard will visit <code><em>41</em></code> distinct positions on your map.</p>
<p>Predict the path of the guard. <em>How many distinct positions will the guard visit before leaving the mapped area?</em></p>
</article>


In [34]:
tests = [
    {
        "name": "Example",
        "s": """
                ....#.....
                .........#
                ..........
                ..#.......
                .......#..
                ..........
                .#..^.....
                ........#.
                #.........
                ......#...
        """,
        "expected": 41,
    },
]


class Map(Str):
    GUARD = {(-1, 0): "^", (0, 1): ">", (1, 0): "v", (0, -1): "<"}

    def __init__(self, s: str) -> None:
        self.m = [list(line.strip()) for line in s.strip().splitlines()]
        self.rows, self.cols = len(self.m), len(self.m[0])
        self.guard = self._guard()

    def turn_right(self, dr, dc) -> tuple[int, int]:
        #  (dr,dc) -> (dc,-dr).
        return dc, -dr

    def count_guard_visited(self) -> int:
        visited = self.guard_visited()
        return len(visited)

    def guard_visited(self) -> set[tuple[int, int]]:
        r, c = self.guard
        dr, dc = -1, 0
        visited = {(r, c)}

        while 0 <= r + dr < self.rows and 0 <= c + dc < self.rows:
            if self.m[r + dr][c + dc] == "#":
                dr, dc = self.turn_right(dr, dc)
                self.m[r][c] = self.GUARD[(dr, dc)]
            else:
                self.m[r][c], self.m[r + dr][c + dc] = (
                    self.m[r + dr][c + dc],
                    self.m[r][c],
                )
                r, c = r + dr, c + dc

            visited.add((r, c))
        return visited

    def _guard(self) -> tuple[int, int]:
        for r, c in product(range(self.rows), range(self.cols)):
            if self.m[r][c] == "^":
                return r, c

        raise ValueError("No guard found")

    def __str__(self) -> str:
        return "\n".join("".join(row) for row in self.m)


@test(tests=tests)
def partI_test(s: str) -> int:
    m = Map(s)
    return m.count_guard_visited()


Test Example passed, for partI_test.
Success


In [35]:
with open("../input/day6.txt") as f:
    puzzle = f.read()

print(f"Part I: {Map(puzzle).count_guard_visited()}")

Part I: 5212


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>5212</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>While The Historians begin working around the guard's patrol route, you borrow their fancy device and step outside the lab. From the safety of a supply closet, you time travel through the last few months and <a href="/2018/day/4">record</a> the nightly status of the lab's guard post on the walls of the closet.</p>
<p>Returning after what seems like only a few seconds to The Historians, they explain that the guard's patrol area is simply too large for them to safely search the lab without getting caught.</p>
<p>Fortunately, they are <em>pretty sure</em> that adding a single new obstruction <em>won't</em> cause a time paradox. They'd like to place the new obstruction in such a way that the guard will get <span title="This vulnerability was later fixed by having the guard always turn left instead."><em>stuck in a loop</em></span>, making the rest of the lab safe to search.</p>
<p>To have the lowest chance of creating a time paradox, The Historians would like to know <em>all</em> of the possible positions for such an obstruction. The new obstruction can't be placed at the guard's starting position - the guard is there right now and would notice.</p>
<p>In the above example, there are only <code><em>6</em></code> different positions where a new obstruction would cause the guard to get stuck in a loop. The diagrams of these six situations use <code>O</code> to mark the new obstruction, <code>|</code> to show a position where the guard moves up/down, <code>-</code> to show a position where the guard moves left/right, and <code>+</code> to show a position where the guard moves both up/down and left/right.</p>
<p>Option one, put a printing press next to the guard's starting position:</p>
<pre><code>....#.....
....+---+#
....|...|.
..#.|...|.
....|..#|.
....|...|.
.#.<em>O</em>^---+.
........#.
#.........
......#...
</code></pre>
<p>Option two, put a stack of failed suit prototypes in the bottom right quadrant of the mapped area:</p><p>
</p><pre><code>....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
......<em>O</em>.#.
#.........
......#...
</code></pre>
<p>Option three, put a crate of chimney-squeeze prototype fabric next to the standing desk in the bottom right quadrant:</p>
<pre><code>....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
.+----+<em>O</em>#.
#+----+...
......#...
</code></pre>
<p>Option four, put an alchemical retroencabulator near the bottom left corner:</p>
<pre><code>....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
..|...|.#.
#<em>O</em>+---+...
......#...
</code></pre>
<p>Option five, put the alchemical retroencabulator a bit to the right instead:</p>
<pre><code>....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
....|.|.#.
#..<em>O</em>+-+...
......#...
</code></pre>
<p>Option six, put a tank of sovereign glue right next to the tank of universal solvent:</p>
<pre><code>....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
.+----++#.
#+----++..
......#<em>O</em>..
</code></pre>
<p>It doesn't really matter what you choose to use as an obstacle so long as you and The Historians can put it into position without the guard noticing. The important thing is having enough options that you can find one that minimizes time paradoxes, and in this example, there are <code><em>6</em></code> different positions you could choose.</p>
<p>You need to get the guard stuck in a loop by adding a single new obstruction. <em>How many different positions could you choose for this obstruction?</em></p>
</article>


In [36]:
from collections.abc import Iterator
from tabulate import tabulate


tests = [
    {
        "name": "Example",
        "s": """
                ....#.....
                .........#
                ..........
                ..#.......
                .......#..
                ..........
                .#..^.....
                ........#.
                #.........
                ......#...
        """,
        "expected": 6,
    },
]


class MapII(Map):
    def guard_visited_with_dir(self) -> Iterator[tuple[int, int, int, int]]:
        r, c = self.guard
        dr, dc = -1, 0

        while 0 <= r < self.rows and 0 <= c < self.rows:
            yield r, c, dr, dc

            if (
                0 <= r + dr < self.rows
                and 0 <= c + dc < self.cols
                and self.m[r + dr][c + dc] == "#"
            ):
                dr, dc = self.turn_right(dr, dc)
            else:
                r, c = r + dr, c + dc

    def place_obstacles(self) -> int:
        count = 0
        seen = set()
        for r, c, dr, dc in self.guard_visited_with_dir():
            if (r + dr, c + dc) in seen:
                continue
            seen.add((r + dr, c + dc))
            if self.check_loop(r, c, dr, dc):
                count += 1

        return count

    def check_loop(self, r: int, c: int, dr: int, dc: int) -> bool:
        blockade_r, blockade_c = r + dr, c + dc

        if (blockade_r, blockade_c) == self.guard:
            return False

        if not (0 <= blockade_r < self.rows and 0 <= c + dc < self.cols):
            return False

        if self.m[blockade_r][blockade_c] == "#":
            return False

        old = self.m[blockade_r][blockade_c]
        self.m[blockade_r][blockade_c] = "#"

        seen = set()

        while 0 <= r + dr < self.rows and 0 <= c + dc < self.cols:

            if (r + dr, c + dc, dr, dc) in seen:
                self.m[blockade_r][blockade_c] = old
                return True

            if self.m[r + dr][c + dc] == "#":
                dr, dc = self.turn_right(dr, dc)
            else:
                r, c = r + dr, c + dc

            seen.add((r, c, dr, dc))

        self.m[blockade_r][blockade_c] = old

        return False


@test(tests=tests)
def partI_test(s: str) -> int:
    m = MapII(s)
    return m.place_obstacles()


Test Example passed, for partI_test.
Success


<link href="style.css" rel="stylesheet"></link>


In [37]:
print(f"Part II: {MapII(puzzle).place_obstacles()}")

Part II: 1767


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1767</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
